In [2]:
running = None

In [ ]:
# Install required packages (run this cell once)
!pip install flask pyngrok pandas -q

from flask import Flask, render_template_string, jsonify, request
from pyngrok import ngrok
import threading
import pandas as pd
from IPython.display import display, HTML

# Global data store for widget data
widget_data = []
# Global dictionary: tracked_vars[name] = current_value
tracked_vars = {}

# Create a basic Flask application
app = Flask(__name__)

@app.route("/")
def home():
    # HTML template includes JavaScript to:
    # 1. Poll the /widgets endpoint for updated data.
    # 2. Provide a "Delete" button for each widget that calls the deleteWidget() function.
    html = """
    <!doctype html>
    <html>
      <head>
        <title>Widget Dashboard</title>
        <script>
          // Function to fetch widget data from the Flask backend
          async function fetchWidgets() {
            try {
              const response = await fetch('/widgets');
              const data = await response.json();
              let htmlContent = '';
              if (data.length === 0) {
                htmlContent = '<p>No widgets available. Please add some via add_widget(key, value)s</p>';
              } else {
                data.forEach(function(widget) {
                  htmlContent += `
                    <div style="border:1px solid #ccc; margin:10px; padding:10px;">
                      <h3>${widget.title}</h3>
                      <div>${widget.content}</div>
                      <button onclick="deleteWidget('${widget.title}')">Delete</button>
                    </div>`;
                });
              }
              document.getElementById("widgets").innerHTML = htmlContent;
            } catch (error) {
              console.error('Error fetching widget data:', error);
            }
          }
          
          // Poll the /widgets endpoint every 5 seconds
          setInterval(fetchWidgets, 5000);
          // Fetch immediately when the page loads
          window.onload = fetchWidgets;
          
          // Function to call the backend to delete a widget by title
          async function deleteWidget(title) {
            try {
              const response = await fetch('/delete_widget', {
                method: 'POST',
                headers: {
                  'Content-Type': 'application/json'
                },
                body: JSON.stringify({title: title})
              });
              const result = await response.json();
              if (result.status === 'success') {
                // Refresh widgets after deletion
                fetchWidgets();
              } else {
                console.error('Failed to delete widget:', title);
              }
            } catch (error) {
              console.error('Error deleting widget:', error);
            }
          }
        </script>
      </head>
      <body>
        <h1>Widget Dashboard</h1>
        <div id="widgets">
          <p>Loading widgets...</p>
        </div>
      </body>
    </html>
    """
    return render_template_string(html)

@app.route("/widgets")
def get_widgets():
    # Return the widget data as JSON
    return jsonify(widget_data)

@app.route("/delete_widget", methods=["POST"])
def delete_widget():
    # Get the widget title from the JSON body of the request
    data = request.get_json()
    title = data.get("title")
    global widget_data
    # Remove any widget with the matching title
    widget_data = [w for w in widget_data if w["title"] != title]
    print(f"Deleted widget: {title}")
    return jsonify({"status": "success"})

@app.route('/shutdown', methods=['POST'])
def shutdown():
    shutdown_func = request.environ.get('werkzeug.server.shutdown')
    if shutdown_func is None:
        raise RuntimeError("Not running with the Werkzeug Server")
    shutdown_func()
    return "Server shutting down..."


def add_widget(title, content):
    """
    Adds or updates a widget with a given title and content.
    If content is a pandas DataFrame, it will be converted to an HTML table.
    If a widget with the same title already exists, its content is updated.
    """
    # Check if content is a pandas DataFrame
    if isinstance(content, pd.DataFrame):
        # Convert the DataFrame to an HTML table.
        content = content.to_html(classes="dataframe", index=True)
    
    # Update widget if exists, otherwise add new widget.
    for widget in widget_data:
        if widget["title"] == title:
            widget["content"] = content
            print(f"Updated widget: {title}")
            break
    else:
        widget_data.append({"title": title, "content": content})
        print(f"Added widget: {title}")

# Set your ngrok API key (replace with your actual API key)
ngrok.set_auth_token("2e4LedVRRoitkUc4V5F4p_59Lsb6SzVniZF7W241VZ2")

# Open an ngrok tunnel on port 5000
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)
display(HTML(f"<a href='{public_url}' target='_blank'>{public_url}</a>"))

def run_app():
    # Run the Flask app on port 5000 without the reloader (to avoid threading issues)
    app.run(port=5000, use_reloader=False, debug=false)

if running:
    shutdown()
    running.join()
    running = None
# Start the Flask app in a background thread.
thread = threading.Thread(target=run_app)
thread.daemon = True  # Ensures the thread stops when the kernel shuts down.
running = thread.start()

print("Flask app is running in the background. You can now call add_widget() to add or update widgets.")


Public URL: https://c7c0-153-242-124-26.ngrok-free.app                                              


Flask app is running in the background. You can now call add_widget() to add or update widgets. * Serving Flask app '__main__'

 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Feb/2025 19:46:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:46:42] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:46:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Feb/2025 19:46:46] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:46:51] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:46:56] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:01] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:06] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:11] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:16] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:21] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:26] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:31] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:47:36] "GET /widgets HTTP/1.1" 200 -
127.0.0.

Deleted widget: some


127.0.0.1 - - [17/Feb/2025 19:48:34] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:48:36] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:48:41] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:48:46] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:48:51] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:48:56] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:01] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:06] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:11] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:16] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:21] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:26] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:31] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:36] "GET /widgets HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 19:49:42] "GET /widgets HTTP/1.1" 2

In [4]:
add_widget("some", "test")

Added widget: some
